In [1]:
import pandas as pd
import spacy
import os
import matplotlib.pyplot as plt

In [2]:
nlp_pt = spacy.load('pt_core_news_sm', exclude=['parser', 'ner'])
portuguese_tagger_pipeline = spacy.load('pt_dual_upos2usas_contextual')
nlp_pt.add_pipe('pymusas_rule_based_tagger', source=portuguese_tagger_pipeline)

nlp_en = spacy.load('en_core_web_sm', exclude=['parser', 'ner'])
english_tagger_pipeline = spacy.load('en_dual_none_contextual')
nlp_en.add_pipe('pymusas_rule_based_tagger', source=english_tagger_pipeline)

nlp_es = spacy.load('es_core_news_sm', exclude=['parser', 'ner'])
spanish_tagger_pipeline = spacy.load('es_dual_upos2usas_contextual')
nlp_es.add_pipe('pymusas_rule_based_tagger', source=spanish_tagger_pipeline)

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/lib/python3.11/site-packages/pymusas/spacy_api/utils.py:38: UserWarning: Overwritten the spaCy Token extension `pymusas_tags` which currently has the following (default, method, getter, setter):`(None, None, None, None)`. And replacing it with the following:`(None, None, None, None)`. This would only become a problem if the the two Tuples of four are different, if they are the same there is no problem.
  warnings.warn(message)
/opt/homebrew/lib/python3.11/site-packages/pymusas/spacy_api/utils.py:38: UserWarning: Overwritten the spaCy Token extension `pymusas_mwe_indexes` which currently has the following (default, method, getter, setter):`(None, None, None, None)`. And replacing it with the following:`(None, None, None, None)`. 

In [3]:
# Funções para processar cada texto
def process_text(text):
    output_doc = nlp_en(str(text))
    result = []
    for token in output_doc:
        result.append({
        'Text': token.text,
        'Lemma': token.lemma_,
        'POS': token.pos_,
        'USAS Tags': token._.pymusas_tags
        })
    return result

def process_text(text):
    output_doc = nlp_es(str(text))
    result = []
    for token in output_doc:
        result.append({
        'Text': token.text,
        'Lemma': token.lemma_,
        'POS': token.pos_,
        'USAS Tags': token._.pymusas_tags
        })
    return result

def process_text(text):
    output_doc = nlp_pt(str(text))
    result = []
    for token in output_doc:
        result.append({
        'Text': token.text,
        'Lemma': token.lemma_,
        'POS': token.pos_,
        'USAS Tags': token._.pymusas_tags
        })
    return result